In [77]:
import dask.dataframe as dd
import pandas as pd
import numpy as np
from dask_ml.xgboost import XGBRegressor
from dask_ml.preprocessing import DummyEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score
import pathlib

In [78]:
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor

In [79]:
data = pd.read_csv('./Data/SUVData.csv')
data.drop(columns='Unnamed: 0', axis=1, inplace=True)

y = data.price
X = data.drop(['price'], axis=1)


In [80]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2)

In [81]:
s = (X.dtypes == 'object')
object_cols = list(s[s].index)

['vin', 'body_type', 'city', 'engine_cylinders', 'exterior_color', 'franchise_make', 'fuel_tank_volume', 'fuel_type', 'highway_fuel_economy', 'interior_color', 'listed_date', 'listing_color', 'make_name', 'maximum_seating', 'model_name', 'sp_name', 'transmission', 'transmission_display', 'wheel_system', 'wheel_system_display']


In [82]:
from sklearn.metrics import r2_score

numerical_cols = [cname for cname in X_train.columns if X_train[cname].dtype in ['int64', 'float64']]

categorical_cols = [cname for cname in X_train.columns if X_train[cname].nunique() < 10 and 
                        X_train[cname].dtype == "object"]

my_cols = categorical_cols + numerical_cols

X_train_edit = X_train[my_cols].copy()
X_valid_edit = X_valid[my_cols].copy()

numerical_transformer = SimpleImputer(strategy='constant')

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
])

model = RandomForestRegressor(n_estimators=150, random_state=1)

pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', model)])

pipeline.fit(X_train, y_train)

preds = pipeline.predict(X_valid)

r2_score(y_valid, preds)

0.9326631700565141